
# 🎓 Introduction to Gradio

Gradio is an open-source Python library that enables you to quickly create user-friendly web interfaces for your machine learning models or any Python functions. It supports various input and output components, such as text, images, and audio, allowing for interactive demonstrations and applications.

---

## 📦 Installation

Before we begin, ensure you have Gradio installed:

```bash
pip install gradio
```

---

## 🧠 Understanding Gradio's Core Concepts

- **Interface**: The primary class in Gradio used to create simple interfaces.
- **Components**: Building blocks like `Textbox`, `Image`, and `Audio` that serve as inputs and outputs.
- **Function (`fn`)**: The Python function that processes inputs and returns outputs.
- **Launch**: Method to start the Gradio app and open it in a web browser.

---

## 📝 Text Input and Output

Let's start with a simple example that takes text input and returns a greeting.

```python
import gradio as gr

def greet(name):
    return f"Hello, {name}!"

demo = gr.Interface(fn=greet, inputs=gr.Textbox(label="Enter your name"), outputs=gr.Textbox(label="Greeting"))
demo.launch()
```

**Explanation**:

- `gr.Textbox`: Creates a text input/output component.
- The `greet` function takes the input text and returns a greeting message.

---

## 🖼️ Image Input and Output

Now, let's create an interface that processes an image. For demonstration, we'll convert an image to grayscale.

```python
import gradio as gr
from PIL import Image
import numpy as np

def to_grayscale(image):
    return image.convert("L")

demo = gr.Interface(fn=to_grayscale, inputs=gr.Image(type="pil", label="Upload Image"), outputs=gr.Image(type="pil", label="Grayscale Image"))
demo.launch()
```

**Explanation**:

- `gr.Image`: Handles image input/output. The `type="pil"` parameter ensures the image is processed as a PIL Image object.
- The `to_grayscale` function converts the uploaded image to grayscale.

---

## 🔊 Audio Input and Output

Let's build an interface that takes an audio input and returns the same audio. This example can be extended to include audio processing like noise reduction or speech recognition.

```python
import gradio as gr

def echo(audio):
    return audio

demo = gr.Interface(fn=echo, inputs=gr.Audio(source="upload", type="filepath", label="Upload Audio"), outputs=gr.Audio(label="Output Audio"))
demo.launch()
```

**Explanation**:

- `gr.Audio`: Manages audio input/output. `source="upload"` allows users to upload audio files.
- The `echo` function returns the same audio without modification.

---

## 🎛️ Combining Multiple Inputs and Outputs

Gradio allows combining multiple input and output components. Here's an example that takes text and an image, and returns a modified image with the text overlayed.

```python
import gradio as gr
from PIL import Image, ImageDraw, ImageFont

def overlay_text(image, text):
    draw = ImageDraw.Draw(image)
    
    # Get image dimensions
    width, height = image.size
    
    # Calculate font size based on image dimensions (e.g., 5% of the smaller dimension)
    font_size = int(min(width, height) * 0.05)
    
    # Use a TrueType font with adjustable size
    try:
        font = ImageFont.truetype("arial.ttf", font_size)
    except IOError:
        # Fallback to default if arial is not available
        font = ImageFont.load_default()
    
    # Get text size (using getbbox or getsize depending on Pillow version)
    try:
        # For newer Pillow versions
        left, top, right, bottom = font.getbbox(text)
        text_width = right - left
        text_height = bottom - top
    except AttributeError:
        # For older Pillow versions
        text_width, text_height = draw.textsize(text, font=font)
    
    # Calculate center position
    x = (width - text_width) // 2
    y = (height - text_height) // 2
    
    # Draw text at center
    draw.text((x, y), text, font=font, fill="red")
    return image

demo = gr.Interface(
    fn=overlay_text,
    inputs=[gr.Image(type="pil", label="Upload Image"), gr.Textbox(label="Enter Text")],
    outputs=gr.Image(type="pil", label="Image with Text")
)
demo.launch()
```

**Explanation**:

- Multiple inputs are passed as a list to the `inputs` parameter.
- The `overlay_text` function draws the input text onto the uploaded image.

---

## 🚀 Sharing Your Gradio App

To share your Gradio app with others, set the `share` parameter to `True` in the `launch` method:

```python
demo.launch(share=True)
```

This will generate a public URL that you can share, allowing others to interact with your app without needing to run the code locally.

---

## 📚 Additional Resources

For more information and advanced usage, refer to the official Gradio documentation: [https://www.gradio.app/docs](https://www.gradio.app/docs)

---




# 🎛️ Gradio Layout with `Blocks`, `Row`, and `Column`

---

## 🧱 What Are Blocks?

Gradio’s `Blocks` API lets you:

- Combine multiple inputs/outputs in a **custom layout**
- Use components in **rows** and **columns**
- Update output **interactively** with buttons, sliders, events, etc.

> Think of it like building a GUI with Lego blocks!

---

## ✅ Basic Structure

```python
import gradio as gr

with gr.Blocks() as demo:
    with gr.Row():
        gr.Textbox(label="Left")
        gr.Textbox(label="Right")
    with gr.Column():
        gr.Button("Button 1")
        gr.Button("Button 2")

demo.launch()
```

---

## 🧪 Full Example: Text + Image + Audio in Row/Column Layout

```python
import gradio as gr
from PIL import Image, ImageDraw, ImageFont
import numpy as np

def process(text, image, audio):
    # Put text on image
    draw = ImageDraw.Draw(image)
    draw.text((10, 10), text, fill="white")
    return image, audio, f"Echo: {text}"

with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Multi-Modal Interface: Text, Image, Audio")

    with gr.Row():
        with gr.Column():
            txt = gr.Textbox(label="Enter text")
            img = gr.Image(type="pil", label="Upload image")
            audio = gr.Audio(source="upload", type="filepath", label="Upload audio")
            btn = gr.Button("Run")

        with gr.Column():
            out_img = gr.Image(label="Image with Text")
            out_audio = gr.Audio(label="Returned Audio")
            out_text = gr.Textbox(label="Text Output")

    btn.click(fn=process, inputs=[txt, img, audio], outputs=[out_img, out_audio, out_text])

demo.launch()
```

#### Another Example:

```python
import gradio as gr
import pandas as pd
import random

# 🎯 Fake Voice Activity Detection (VAD) Function
def voice_activity_detection(audio_path):
    # Generate random start and end times to simulate detected speech segments
    starts = [round(random.uniform(0, 10), 2) for _ in range(3)]
    ends = [start + round(random.uniform(0.5, 2), 2) for start in starts]

    # Create a DataFrame to display results
    df = pd.DataFrame({
        "start": starts,
        "end": ends
    })
    return df

# 🧱 Gradio Interface with Layout (Blocks, Rows, and Columns)
with gr.Blocks() as demo:
    gr.Markdown("## 🎤 Voice Activity Detection (Fake Example)")

    with gr.Row():
        with gr.Column():
            # Input audio component (uploaded from user)
            audio_input = gr.Audio(source="upload", type="filepath", label="Upload audio file")

            # Button to trigger the processing function
            btn = gr.Button("Run VAD")

        with gr.Column():
            # Output table (DataFrame) to display VAD segments
            output_df = gr.Dataframe(headers=["start", "end"], label="Detected Speech Segments")

    # Bind button click to the VAD function
    btn.click(fn=voice_activity_detection, inputs=audio_input, outputs=output_df)

# Launch the interface
demo.launch()
```

---

## 📐 Layout Guide

| Component | Description |
|-----------|-------------|
| `gr.Blocks()` | Container for the full layout |
| `gr.Row()`    | Horizontal container (left to right) |
| `gr.Column()` | Vertical container (top to bottom) |
| Nesting      | You can nest rows inside columns and vice versa |

---

## ✨ Bonus: Add Header/Footer with `gr.Markdown`

```python
gr.Markdown("# 🌟 My Fancy Gradio App")
```

---

## 🧠 Summary

| Feature       | Benefit                                |
|---------------|-----------------------------------------|
| `Blocks`      | Full layout control                     |
| `Row`         | Place components horizontally           |
| `Column`      | Place components vertically             |
| `.click()`    | Attach logic to buttons and events      |
| Reusable      | Great for dashboards and complex UIs    |

